In [1082]:
import pandas as pd

def read_spenvis(file):
    lines = open(file).read().split('\n')
    data = []
    new = {}
    start = 0
    skips = 0
    headers = None
    for i,line in enumerate(lines):
        if skips > 0: skips -= 1; continue
        line = line.replace('\'','')
        if line.startswith('*'): start = i + int(line.split(',')[1].split(',')[0])
        elif line.startswith('SPENVIS'): new['META'] = line
        elif line.startswith('PS Annotation'): skips = int(line.split(',')[1]); headers = []
        elif i < start: 
            if headers is not None: 
                if line.split(',')[-1] in new.keys():
                    new['SPECTRUM'] = [f"{new['MOD_ABB']} {x} {new[line.split(',')[-1]][-1]}" for x in new[line.split(',')[-1]][:-1]]
                    headers += new['SPECTRUM']
                else: headers.append(f"{line.split(',')[0]} ({line.split(',')[1].strip()})")
            else:
                if line.split(',')[1].strip() == '-1': new[line.split(',')[0]] = line.split(',')[2]
                elif line.split(',')[1].strip() == '1': 
                    try: new[line.split(',')[0]] = float(line.split(',')[2])
                    except ValueError: new[line.split(',')[0]] = line.split(',')[2]
                else: new[line.split(',')[0]] = [x.strip() for x in line.split(',')[2:]]
        elif line in ['End of Block','End of File']:
            # print('Reading',file.split('/')[-1],'table',len(data),'lines',start,i,new['PLT_HDR'] if 'PLT_HDR' in new.keys() else '')
            new['DF'] = pd.read_csv(file,skiprows=start,nrows=i-start,header=None)
            try: new['DF'].columns = headers
            except ValueError:
                if 'PLT_LEG' in new.keys(): new['DF'].columns = headers[:-1] + [x.strip() for x in new['PLT_LEG']]
            headers = None
            data.append(new)
            new = {}
    return data

def get(spenvis,segment,shielding='0.100 mm'):
    orbit = read_spenvis(spenvis + '/spenvis_sao.txt')[segment]
    orbit['DF']['Segment'] = orbit['ORB_HDR']
    orbit['DF']['Time (hrs)'] = (orbit['DF']['MJD (days)'] - orbit['DF']['MJD (days)'].min()) * 24
    
    niel = read_spenvis(spenvis + '/spenvis_nio.txt')

    tfluence = niel[segment]
    atten = tfluence['DF'][shielding] / tfluence['DF']['Unshielded']
    tflux = read_spenvis(spenvis + '/spenvis_spp.txt')[segment]
    tflux['DF'][tflux['SPECTRUM']] *= atten.values

    atten = niel[-4]['DF'][shielding] / niel[-4]['DF']['Unshielded']
    sfluence = read_spenvis(spenvis + '/spenvis_sef.txt')[0]
    sflux = read_spenvis(spenvis + '/spenvis_seo.txt')[segment]
    sflux['DF'] *= list(sfluence['DF']['IFlux (cm!u-2!n)'] / sfluence['DF']['Attenuation ()'] * atten / (orbit['MIS_DUR']*24*3600))

    gcr = read_spenvis(spenvis + '/spenvis_nlof_srimsi.txt')[3 * segment]
    gcr['DF'] = gcr['DF'].pivot(columns='LET (MeV cm!u2!n g!u-1!n)',values='IFlux (m!u-2!n sr!u-1!n s!u-1!n)').fillna(method='bfill').iloc[0]
    gcr['DF'] = pd.concat([gcr['DF']] * len(orbit['DF']),axis=1).T.reset_index(drop=True)
    gcr['DF'].columns = ['GCR ' + str(x) + ' LET' for x in gcr['DF'].columns]
    gcr['SPECTRUM'] = gcr['DF'].columns

    pstar = pd.read_csv('pstar.txt',sep=' ',skiprows=8,header=None)
    pstar.columns = ['Energy','LET',2]
    pstar = pstar.pivot(columns='Energy',values='LET').fillna(method='bfill').iloc[0]
    tlet = pstar[[float(x) for x in tflux['ENERGY'][:-1]]].values
    ttid = tflux['DF'][tflux['SPECTRUM']].cumsum().mul(tlet).sum(axis=1) * 1.6e-8
    slet = pstar[[float(x) for x in sflux['ENERGY'][:-1]]].values
    stid = sflux['DF'][sflux['SPECTRUM']].cumsum().mul(slet).sum(axis=1) * 1.6e-8
    tid = pd.DataFrame({'TTID':ttid,'STID':stid,'TID':ttid + stid})

    scale = niel[0]['NIE_RCT']
    damage = [scale * float(x) for x in tflux['ENERGY'][:-1]]
    tddd = tflux['DF'][tflux['SPECTRUM']].cumsum().mul(damage).sum(axis=1)
    damage = [scale * float(x) for x in sflux['ENERGY'][:-1]]
    sddd = sflux['DF'][sflux['SPECTRUM']].cumsum().mul(damage).sum(axis=1)
    ddd = pd.DataFrame({'TDDD':tddd,'SDDD':sddd,'DDD':tddd + sddd})

    df = pd.concat([orbit['DF'],tflux['DF'],sflux['DF'],gcr['DF'],tid,ddd],axis=1)
    df.to_csv(f'trajectory/segment{segment}.csv',index=False)
    return df,(orbit,tflux,sflux,gcr,tid,ddd)

segment = 2
df,(orbit,tflux,sflux,gcr,tid,ddd) = get('trajectory/SPENVIS',segment)
print(df)

       MJD (days)  Altitude (km)  Latitude (deg)  Longitude (deg)  \
0    26722.000000       35778.84    0.000000e+00       202.392898   
1    26722.002778       35778.83   -2.761378e-08       202.393157   
2    26722.005556       35778.83   -1.063376e-07       202.393417   
3    26722.008333       35778.83   -2.299405e-07       202.393676   
4    26722.011111       35778.83   -3.921210e-07       202.393935   
..            ...            ...             ...              ...   
354  26722.983333       35778.63   -3.783483e-03       202.502296   
355  26722.986111       35778.64   -3.800799e-03       202.502566   
356  26722.988889       35778.65   -3.817104e-03       202.502835   
357  26722.991667       35778.66   -3.832387e-03       202.503104   
358  26722.994444       35778.67   -3.846639e-03       202.503372   

     LocalTime (hrs)  Alpha (deg) Segment  Time (hrs)  B (Gauss)  \
0          13.492860         90.0     GEO    0.000000   0.001082   
1          13.559544         90.0  

In [1083]:
from plotly.subplots import make_subplots
def vertical_subplots(figs,sharex=True):
    fig = make_subplots(rows=len(figs),shared_xaxes=sharex)
    for i in range(len(figs)):
        if len(figs[i].data) == 1: figs[i].update_traces(name=figs[i].layout.yaxis.title.text,showlegend=True)
        for trace in figs[i].update_traces(legendgroup=i).select_traces(): fig.add_trace(trace,row=i+1,col=1)
        fig.update_yaxes(type=figs[i].layout.yaxis.type,row=i+1,col=1)
        fig.update_xaxes(type=figs[i].layout.xaxis.type,row=i+1,col=1)
    return fig

default = {
    'margin':{'l':80,'r':80,'t':50,'b':50},
    'template':'plotly_white',
    'paper_bgcolor':'rgba(0,0,0,0)',
    'plot_bgcolor':'rgba(0,0,0,0)',
    'showlegend':False
}

vertical_subplots([
    df.plot('Time (hrs)','Altitude (km)'),
    df.plot('Time (hrs)','Latitude (deg)'),
    df.plot('Time (hrs)','Longitude (deg)'),
    df.plot('Time (hrs)',['TRP 1.00 MeV','TRP 10.00 MeV','TRP 100.00 MeV'],log_y=True),
    df.plot('Time (hrs)',['SEP 1.00 MeV','SEP 10.00 MeV','SEP 100.00 MeV'],log_y=True),
    df.plot('Time (hrs)',['GCR 1.609 LET','GCR 498.59 LET','GCR 40145.0 LET'],log_y=True),
    df.plot('Time (hrs)','TID'),
    df.plot('Time (hrs)','DDD')
]).update_layout(default,height=800,width=500).write_image(f'trajectory/segment{segment}_time.png',scale=3)

vertical_subplots([
    df.plot(x=[float(x) for x in tflux['ENERGY'][:-1]],y=df[tflux['SPECTRUM']].mean(),log_y=True,log_x=True),
    df.plot(x=[float(x) for x in sflux['ENERGY'][:-1]],y=df[sflux['SPECTRUM']].mean(),log_y=True,log_x=True),
    df.plot(x=[float(x.split(' ')[1]) for x in gcr['SPECTRUM']],y=df[gcr['SPECTRUM']].mean(),log_y=True,log_x=True)
],sharex=False).update_layout(default,height=400,width=500).write_image(f'trajectory/segment{segment}_energy.png',scale=3)

In [752]:

G = 6.6743e-11 # m3/kg/s2
m_earth = 5.97219e24 # kg
r_earth = 6371 # km
def ellipse(peri,apo,incl,angle):
    peri = peri + r_earth
    apo = apo + r_earth
    semi = (peri + apo) / 2
    ecc = (apo - peri) / (apo + peri)
    theta = np.linspace(0, 2 * np.pi, 1000) + [0]
    r = semi * (1 - ecc ** 2) / (1 + ecc * np.cos(theta))
    x = r * np.cos(theta) * np.cos(np.radians(incl))
    y = r * np.sin(theta)
    z = r * np.sin(np.radians(incl)) * np.cos(theta)
    v = np.sqrt(G * m_earth / (r ** 2))
    return x,y,z,v

import plotly.express as px
import plotly.graph_objects as go

x,y,z,v = ellipse(35786,35786,0,0)
fig = go.Figure(go.Scatter(x=x,y=y,mode='markers',marker={
    'color':v,'colorscale':'BlueRed','size':4,'cmin':450,'cmax':2750}))

xe,ye,ze,ve = ellipse(0,0,0,0)
path='M '+str(xe[0])+','+str(ye[1])
for k in range(1, xe.shape[0]):
     path+=' L '+str(xe[k])+','+str(ye[k])
path+=' Z'

layout = {
    'margin':{'l':80,'r':80,'t':50,'b':50},
    'template':'plotly_white',
    'paper_bgcolor':'rgba(0,0,0,0)',
    'plot_bgcolor':'rgba(0,0,0,0)',
    'showlegend':False,
}
fig.update_layout(layout,xaxis={'range':[-36000,36000]},yaxis={'scaleanchor':'x','scaleratio':1},
    shapes=[dict(type='path',layer='below',path=path,fillcolor='cornflowerblue')],
    height=1200,width=1200
).update_xaxes(visible=False,showgrid=False).update_yaxes(visible=False,showgrid=False)
fig.write_image(f'trajectory/GEO_orbit.png',scale=3)
fig.show()